In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
data=pd.read_csv("mushrooms.csv")

In [3]:
data.head(5)

,type,cap_shape,cap_surface,cap_color,bruises,odor,gill_attachment,gill_spacing,gill_size,gill_color,...,stalk_surface_below_ring,stalk_color_above_ring,stalk_color_below_ring,veil_type,veil_color,ring_number,ring_type,spore_print_color,population,habitat
0,p,x,s,n,t,p,f,c,n,k,...,s,w,w,p,w,o,p,k,s,u
1,e,x,s,y,t,a,f,c,b,k,...,s,w,w,p,w,o,p,n,n,g
2,e,b,s,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,n,m
3,p,x,y,w,t,p,f,c,n,n,...,s,w,w,p,w,o,p,k,s,u
4,e,x,s,g,f,n,f,w,b,k,...,s,w,w,p,w,o,e,n,a,g


In [5]:
x=list(data.columns)
print(x)

['type', 'cap_shape', 'cap_surface', 'cap_color', 'bruises', 'odor', 'gill_attachment', 'gill_spacing', 'gill_size', 'gill_color', 'stalk_shape', 'stalk_root', 'stalk_surface_above_ring', 'stalk_surface_below_ring', 'stalk_color_above_ring', 'stalk_color_below_ring', 'veil_type', 'veil_color', 'ring_number', 'ring_type', 'spore_print_color', 'population', 'habitat']


In [6]:
for i in x:
    print(data[i].value_counts())

e    4208
p    3916
Name: type, dtype: int64
x    3656
f    3152
k     828
b     452
s      32
c       4
Name: cap_shape, dtype: int64
y    3244
s    2556
f    2320
g       4
Name: cap_surface, dtype: int64
n    2284
g    1840
e    1500
y    1072
w    1040
b     168
p     144
c      44
r      16
u      16
Name: cap_color, dtype: int64
f    4748
t    3376
Name: bruises, dtype: int64
n    3528
f    2160
s     576
y     576
a     400
l     400
p     256
c     192
m      36
Name: odor, dtype: int64
f    7914
a     210
Name: gill_attachment, dtype: int64
c    6812
w    1312
Name: gill_spacing, dtype: int64
b    5612
n    2512
Name: gill_size, dtype: int64
b    1728
p    1492
w    1202
n    1048
g     752
h     732
u     492
k     408
e      96
y      86
o      64
r      24
Name: gill_color, dtype: int64
t    4608
e    3516
Name: stalk_shape, dtype: int64
b    3776
?    2480
e    1120
c     556
r     192
Name: stalk_root, dtype: int64
s    5176
k    2372
f     552
y      24
Name: stalk_surfa

In [7]:
data.shape

(8124, 23)

## Label Encoding
### OneHotEncoding

In [8]:
d=pd.DataFrame([['a','p'],['b','q'],['c','p'],['b','p'],['c','d'],['a','q'],['b','r']])

In [9]:
d.columns=["f1",'f2']
d

,f1,f2
0,a,p
1,b,q
2,c,p
3,b,p
4,c,d
5,a,q
6,b,r


In [10]:
pd.get_dummies(d,drop_first=True)

,f1_b,f1_c,f2_p,f2_q,f2_r
0,0,0,1,0,0
1,1,0,0,1,0
2,0,1,1,0,0
3,1,0,1,0,0
4,0,1,0,0,0
5,0,0,0,1,0
6,1,0,0,0,1


In [11]:
from sklearn.preprocessing import OneHotEncoder

In [14]:
oht=OneHotEncoder()

In [15]:
oht.fit_transform(d).toarray()

array([[1., 0., 0., 0., 1., 0., 0.],
       [0., 1., 0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 1., 0., 0.],
       [0., 1., 0., 0., 1., 0., 0.],
       [0., 0., 1., 1., 0., 0., 0.],
       [1., 0., 0., 0., 0., 1., 0.],
       [0., 1., 0., 0., 0., 0., 1.]])

In [16]:
oht.transform([['a','q']]).toarray()

array([[1., 0., 0., 0., 0., 1., 0.]])

In [17]:
oht.categories_

[array(['a', 'b', 'c'], dtype=object),
 array(['d', 'p', 'q', 'r'], dtype=object)]

In [18]:
oht.inverse_transform([[1,0,0,0,0,1,0]])

array([['a', 'q']], dtype=object)

## Encoding Categorical Data Into Numeric

In [21]:
from sklearn.preprocessing import LabelEncoder

In [22]:
le=LabelEncoder()

In [23]:
data=data.apply(le.fit_transform)

In [24]:
le.classes_

array(['d', 'g', 'l', 'm', 'p', 'u', 'w'], dtype=object)

In [25]:
le.inverse_transform(data["habitat"])

array(['u', 'g', 'm', ..., 'l', 'l', 'l'], dtype=object)

In [26]:
data=data.values

In [27]:
Y=data[:,0]
X=data[:,1:]

In [28]:
from sklearn.model_selection import train_test_split

In [29]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=.2,random_state=42)

In [30]:
X_train.shape,Y_train.shape

((6499, 22), (6499,))

## Naive Bayes Classifier

In [31]:
def prior_prob(Y_train,label):
    return (Y_train==label).mean()

In [32]:
def conditional_prob(X_train,Y_train,label,f_col,f_val):
    X_des=X_train[Y_train==label]
    return (X_des[:,f_col]==f_val).mean()

In [38]:
## pi()

def Prediction(x_train,y_train,x):
    classes=np.unique(y_train)
    max_posterior=0
    ans_class=0
    
    for y in classes:
        pro=prior_prob(y_train,y)
        
        totalLikelihood=1
        for i in range(x_train.shape[1]):
            totalLikelihood*=conditional_prob(X_train,Y_train,y,i,x[i])
        
        posterior=totalLikelihood*pro
        
        if(posterior>max_posterior):
            ans_class=y
            max_prosterior=posterior
    
    return ans_class

In [36]:
def score(X_train,Y_train,X_test,Y_test):
    Y_pred=[]
    for x in X_test:
        Y_pred.append(Prediction(X_train,Y_train,x))
    return (np.array(Y_pred)==Y_test).mean()
    

In [39]:
score(X_train,Y_train,X_test,Y_test)


0.8178461538461539

In [40]:
from sklearn.naive_bayes import MultinomialNB

In [41]:
mnb=MultinomialNB()

In [42]:
mnb.fit(X_train,Y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [43]:
mnb.score(X_test,Y_test)

0.8073846153846154